In [5]:
import collections
import string
import re

In [8]:
def find_repeated_sequences(ciphertext, min_length=3):
    """
    Finds repeated sequences of characters in the ciphertext.
    
    Returns:
    dict: A dictionary where keys are the repeated sequences and values are lists of their positions.
    """
    repeated_sequences = {}
    for length in range(min_length, len(ciphertext)):
        for start_pos in range(len(ciphertext) - length + 1):
            sequence = ciphertext[start_pos:start_pos + length]
            if sequence in repeated_sequences:
                repeated_sequences[sequence].append(start_pos)
            else:
                repeated_sequences[sequence] = [start_pos]
    return {k: v for k, v in repeated_sequences.items() if len(v) > 1}



In [9]:
def calculate_distances(repeated_sequences):
    """
    Calculates the distances between repeated sequences.

    Args:
    repeated_sequences (dict): Dictionary of repeated sequences and their positions.

    Returns:
    list: List of distances between repeated sequences.
    """
    distances = []
    for sequence, positions in repeated_sequences.items():
        for i in range(len(positions) - 1):
            distance = positions[i + 1] - positions[i]
            distances.append(distance)
    return distances


In [10]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

In [11]:
def find_key_length(ciphertext):
    """
    Finds the estimated length of the repeating key in the Vigenère cipher.

    Returns:
    int: Estimated key length.
    """
    repeated_sequences = find_repeated_sequences(ciphertext)
    distances = calculate_distances(repeated_sequences)
    unique_distances = list(set(distances))
    potential_key_lengths = []
    for distance in unique_distances:
        for other_distance in unique_distances:
            if distance != other_distance:
                potential_key_lengths.append(gcd(distance, other_distance))
    estimated_key_length = max(set(potential_key_lengths), key=potential_key_lengths.count)
    return estimated_key_length

In [ ]:
def vigenere_decrypt(ciphertext, key_length):
    # Create groups based on key length
    groups = ['' for _ in range(key_length)]
    for i, char in enumerate(ciphertext):
        groups[i % key_length] += char

    # Define English letter frequencies
    english_frequencies = {'e': 0.127, 't': 0.091, 'a': 0.082, 'o': 0.075, 'i': 0.070,
                           'n': 0.067, 's': 0.063, 'h': 0.061, 'r': 0.060, 'd': 0.043,
                           'l': 0.040, 'c': 0.028, 'u': 0.028, 'm': 0.024, 'w': 0.024,
                           'f': 0.022, 'g': 0.020, 'y': 0.020, 'p': 0.019, 'b': 0.015,
                           'v': 0.010, 'k': 0.008, 'j': 0.002, 'x': 0.001, 'q': 0.001, 'z': 0.001}

    def chi_squared(observed_frequencies):
        chi2 = 0
        total = sum(observed_frequencies.values())
        for letter, expected_frequency in english_frequencies.items():
            observed_frequency = observed_frequencies[letter] / total
            chi2 += (observed_frequency - expected_frequency) ** 2 / expected_frequency
        return chi2

    # Decrypt each group using frequency analysis
    decrypted_groups = []
    for group in groups:
        # Count letter frequencies in the group
        frequency_counter = collections.Counter(group.lower())
        # Sort by frequency (most frequent first)
        sorted_frequencies = sorted(frequency_counter.items(), key=lambda x: x[1], reverse=True)
        # Assuming the most frequent letter is 'e'
        most_frequent_letter = sorted_frequencies[0][0]
        # Shift to get the key letter
        key_letter = chr((ord(most_frequent_letter) - ord('e')) % 26 + ord('a'))
        # Decrypt the group
        decrypted_group = ''
        for char in group:
            decrypted_char = chr((ord(char) - ord(key_letter)) % 26 + ord('a'))
            decrypted_group += decrypted_char
        decrypted_groups.append(decrypted_group)

    # Combine decrypted groups into the plaintext
    plaintext = ''
    for i in range(len(ciphertext)):
        plaintext += decrypted_groups[i % key_length][i // key_length]

    return plaintext


In [ ]:
ciphertext = ""
with open("output.txt", 'r') as file:
    ciphertext = file.read().replace(' ','')
    
key_length = find_key_length(ciphertext)
print(key_length)